다중 선형회귀 분석

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 선형회귀 관련 라이브러리 가져오기
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm

In [2]:
# 간단한 예제
print(np.array([[0,1],[1,2],[2,2.5]]))
print()
print(np.array([0,1.2,1.6]))

[[0.  1. ]
 [1.  2. ]
 [2.  2.5]]

[0.  1.2 1.6]


In [3]:
# 데이터

x = np.array([[0,1],[1,2],[2,2.5]])
y = np.array( [0,1.2,1.6])

# 선형회귀 object 생성
reg = linear_model.LinearRegression()

#  훈련
reg.fit(x,y)

# training data 예측
pred_train = reg.predict(x)

# test
pred_test = reg.predict([[1.5, 2]])

In [4]:
# test data 예측값
pred_test

array([1.])

In [5]:
# coefficient(기울기, beta1, beta2)
reg.coef_

array([-0.4,  1.6])

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/mulcam_bigdata

/content/drive/MyDrive/mulcam_bigdata


In [8]:
path = './data/Advertising.csv'
ad = pd.read_csv(path, index_col=0)
ad

,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9
...,...,...,...,...
196,38.2,3.7,13.8,7.6
197,94.2,4.9,8.1,9.7
198,177.0,9.3,6.4,12.8
199,283.6,42.0,66.2,25.5


In [9]:
# 선형회귀 모델 적용
ad.shape

(200, 4)

In [10]:
ad.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 1 to 200
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   Radio      200 non-null    float64
 2   Newspaper  200 non-null    float64
 3   Sales      200 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB


In [11]:
# training / test data 분리
train = ad[:-20]  # 180
test = ad[-20:]  # 20

# training data의 feature / response 분리
train_x = train[['TV', 'Radio', 'Newspaper']]
train_y = train[['Sales']]

# test data의 feature / response 분리
test_x = test[['TV', 'Radio', 'Newspaper']]
test_y = test[['Sales']]

In [17]:
# 선형회귀 객체 만들기
lr = linear_model.LinearRegression()

# training data >> 모델 적합
lr.fit(train_x, train_y)

# training data 이용, 예측 (실무: 모덱 적합도, 과적합 여부 판단)
train_y_pred = lr.predict(train_x)

# test data 이용, 예측 (**)
test_y_pred = lr.predict(test_x)


In [18]:
# coefficients
lr.coef_
# 회귀계수 반환
# 이 값들은 각 특성이 예측값에 미치는 영향
# 값이 클수록 해당 특성의 영향력이 크다는 것을 의미

array([[ 0.04638909,  0.18867512, -0.0024597 ]])

In [19]:
# training MSE
mean_squared_error(train_y, train_y_pred)
# 평균 제곱 오차(MSE, Mean Squared Error)를 계산하는 데 사용
# 이 함수는 모델의 예측값과 실제값 사이의 차이를 정량적으로 나타내는 중요한 지표
# MSE 값이 작을수록 모델의 예측이 실제값에 더 가까움
# MSE 값이 클수록 모델의 예측이 실제값과 더 많이 차이남

2.827418881491677

In [20]:
# test MSE
mean_squared_error(test_y, test_y_pred)

2.4528179307176843

In [ ]:
'''
훈련 데이터 MSE(2.827) > 테스트 데이터 MSE(2.453)

- 훈련 데이터의 MSE가 테스트 데이터의 MSE보다 크다는 것은 모델이 훈련 데이터에
  약간 더 부정확
- 이는 일반적으로 바람직하지 않은 현상
- 이상적인 경우 훈련 데이터 MSE와 테스트 데이터 MSE는 비슷해야 함

모델의 일반화 성능

- 일반적으로, 훈련 데이터와 테스트 데이터에 대한 MSE 값이 비슷하다면 모델이
  잘 일반화되었다고 할 수 있음
- 현재의 경우, 훈련 데이터의 MSE가 테스트 데이터의 MSE보다 약간 더 크므로
  모델이 훈련 데이터에 과적합(overfitting)되었다고 판단하기는 어렵움.
- 테스트 데이터의 MSE가 훈련 데이터의 MSE보다 낮은 것은 우연일 수 있으며,
  일반적으로 데이터의 분포나 모델의 복잡도와 관련이 있을 수 있습니다.
'''

In [21]:
# r2_score
r2_score(train_y, train_y_pred)

0.8923555807586847

In [22]:
r2_score(test_y, test_y_pred)
# 과소적합 의심 >> n을 늘려야 함

0.9288231093749743

In [23]:
# statsmodel (sm) 이용
# 반드시 x0 feature 추가해야만 함
train_x

sm_train_x = train_x
sm_train_x['x0'] = 1

sm_test_x = test_x
sm_test_x['x0'] = 1

# x0 특성을 추가하는 이유는 회귀 모델에서 상수 항(절편)을 포함시키기 위해서
# 상수 항이 없으면 회귀 모델은 원점을 지나는 직선으로 제한되어,
# 데이터의 편향을 제대로 학습할 수 없음
# statsmodels의 OLS(Ordinary Least Squares) 모델은 명시적으로 상수 항을 포함시켜야 함.
# x0 특성으로 1을 추가하는 것은 이를 위해 사용됨

# 회귀 분석에서 상수 항(절편)은 독립 변수 𝑋와의 관계를 모델링하기 위한 것
# 즉, 종속 변수 y에 상수 항을 추가하지 않음

In [24]:
sm_train_x

,TV,Radio,Newspaper,x0
1,230.1,37.8,69.2,1
2,44.5,39.3,45.1,1
3,17.2,45.9,69.3,1
4,151.5,41.3,58.5,1
5,180.8,10.8,58.4,1
...,...,...,...,...
176,276.9,48.9,41.8,1
177,248.4,30.2,20.3,1
178,170.2,7.8,35.2,1
179,276.7,2.3,23.7,1


In [25]:
sm_test_x

,TV,Radio,Newspaper,x0
181,156.6,2.6,8.3,1
182,218.5,5.4,27.4,1
183,56.2,5.7,29.7,1
184,287.6,43.0,71.8,1
185,253.8,21.3,30.0,1
186,205.0,45.1,19.6,1
187,139.5,2.1,26.6,1
188,191.1,28.7,18.2,1
189,286.0,13.9,3.7,1
190,18.7,12.1,23.4,1


In [26]:
# training
results = sm.OLS(train_y, sm_train_x).fit()
#  sm.OLS(y, X).fit()의 순서는 y (종속 변수), X (설명 변수)

# training 결과 확인
results.summary()


# OLS는 선형회귀분석의 한 방법론으로서 사용되며,
# 선형회귀분석은 다양한 방법론과 기법을 포함하는 더 넓은 개념

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.891
Method:                 Least Squares   F-statistic:                     486.3
Date:                Mon, 01 Jul 2024   Prob (F-statistic):           6.57e-85
Time:                        14:12:01   Log-Likelihood:                -348.95
No. Observations:                 180   AIC:                             705.9
Df Residuals:                     176   BIC:                             718.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TV             0.0464      0.001     31.154      0.000       0.043       0.049
Radio          0.1887      0.009     20.347      0.000       0.170       0.207
Newspaper     -0.0025      0.006     -0.395      0.693      -0.015       0.010
x0             2.8399      0.342      8.293      0.000       2.164       3.516
==============================================================================
Omnibus:                       56.196   Durbin-Watson:                   2.104
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              140.467
Skew:                          -1.343   Prob(JB):                     3.15e-31
Kurtosis:                       6.394   Cond. No.                         467.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

다항회귀분석

In [27]:
from sklearn.preprocessing import PolynomialFeatures

np.arange(6).reshape(3,2)

x = np.arange(6).reshape(3,2)
x

array([[0, 1],
       [2, 3],
       [4, 5]])

In [28]:
# [1,a,b,a^2, ab, b^2] feature 생성
poly = PolynomialFeatures(2)
poly.fit_transform(x)

#  x가 특성 벡터 [x1, x2, ...]라면,
# PolynomialFeatures(2)는 [1, x1, x2, x1^2, x1*x2, x2^2, ...]와 같은 다항식 특성을 생성
# 1은 PolynomialFeatures에 의해 추가된 상수 항

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

In [29]:
# interaction feature 만 생성하고 싶을 때
poly = PolynomialFeatures(interaction_only=True)
poly.fit_transform(x)
# 1은 PolynomialFeatures에 의해 추가된 상수 항
# x1*x2 만 추가

array([[ 1.,  0.,  1.,  0.],
       [ 1.,  2.,  3.,  6.],
       [ 1.,  4.,  5., 20.]])

auto 데이터 활용, 회귀분석 실습

In [31]:
path ='./data/Auto.csv'
auto = pd.read_csv(path)
auto

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
392,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
393,44.0,4,97.0,52,2130,24.6,82,2,vw pickup
394,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage
395,28.0,4,120.0,79,2625,18.6,82,1,ford ranger


In [33]:
auto.shape
# 397개의 행, 9개의 열

(397, 9)

In [34]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           397 non-null    float64
 1   cylinders     397 non-null    int64  
 2   displacement  397 non-null    float64
 3   horsepower    397 non-null    object 
 4   weight        397 non-null    int64  
 5   acceleration  397 non-null    float64
 6   year          397 non-null    int64  
 7   origin        397 non-null    int64  
 8   name          397 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.0+ KB


In [35]:
auto['horsepower'].unique()
# '?' 값 존재 >> object

array(['130', '165', '150', '140', '198', '220', '215', '225', '190',
       '170', '160', '95', '97', '85', '88', '46', '87', '90', '113',
       '200', '210', '193', '?', '100', '105', '175', '153', '180', '110',
       '72', '86', '70', '76', '65', '69', '60', '80', '54', '208', '155',
       '112', '92', '145', '137', '158', '167', '94', '107', '230', '49',
       '75', '91', '122', '67', '83', '78', '52', '61', '93', '148',
       '129', '96', '71', '98', '115', '53', '81', '79', '120', '152',
       '102', '108', '68', '58', '149', '89', '63', '48', '66', '139',
       '103', '125', '133', '138', '135', '142', '77', '62', '132', '84',
       '64', '74', '116', '82'], dtype=object)

In [36]:
# horsepower '?'값을 '0'으로 대체
auto['horsepower'] = auto['horsepower'].replace('?', '0')

auto 다중선형회귀 (sklearn)

In [37]:
auto[:10]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
5,15.0,8,429.0,198,4341,10.0,70,1,ford galaxie 500
6,14.0,8,454.0,220,4354,9.0,70,1,chevrolet impala
7,14.0,8,440.0,215,4312,8.5,70,1,plymouth fury iii
8,14.0,8,455.0,225,4425,10.0,70,1,pontiac catalina
9,15.0,8,390.0,190,3850,8.5,70,1,amc ambassador dpl


In [40]:
auto = auto.drop(['name'], axis = 1)

In [41]:
auto [:10]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,18.0,8,307.0,130,3504,12.0,70,1
1,15.0,8,350.0,165,3693,11.5,70,1
2,18.0,8,318.0,150,3436,11.0,70,1
3,16.0,8,304.0,150,3433,12.0,70,1
4,17.0,8,302.0,140,3449,10.5,70,1
5,15.0,8,429.0,198,4341,10.0,70,1
6,14.0,8,454.0,220,4354,9.0,70,1
7,14.0,8,440.0,215,4312,8.5,70,1
8,14.0,8,455.0,225,4425,10.0,70,1
9,15.0,8,390.0,190,3850,8.5,70,1


In [42]:
# training / test 데이터 분리

train = auto[:-40]
test = auto[-40:]

# training data 의 feature와 response 분리
train_x = train.iloc[:, 1:]
train_y = train[['mpg']]
print(train_x)
# mpg	열이 종속변수 y

print()
# test data 의 feature와 response 분리
test_x = test.iloc[:, 1:]
test_y = test[['mpg']]
print(test_x)

     cylinders  displacement horsepower  weight  acceleration  year  origin
0            8         307.0        130    3504          12.0    70       1
1            8         350.0        165    3693          11.5    70       1
2            8         318.0        150    3436          11.0    70       1
3            8         304.0        150    3433          12.0    70       1
4            8         302.0        140    3449          10.5    70       1
..         ...           ...        ...     ...           ...   ...     ...
352          4          98.0         65    2380          20.7    81       1
353          4         105.0         74    2190          14.2    81       2
354          4         100.0          0    2320          15.8    81       2
355          4         107.0         75    2210          14.4    81       3
356          4         108.0         75    2350          16.8    81       3

[357 rows x 7 columns]

     cylinders  displacement horsepower  weight  acceleration  

In [43]:
# 선형회귀 객체 생성
lr = linear_model.LinearRegression()

# training data 를 이용 >> 적합
lr.fit(train_x, train_y)

# training data 예측
train_y_pred = lr.predict(train_x)

# test data 예측
test_y_pred = lr.predict(test_x)

In [44]:
lr.coef_

array([[-0.3736576 ,  0.01504545, -0.01430246, -0.00625177,  0.03568412,
         0.74444736,  1.44269524]])

In [45]:
mean_squared_error(train_y, train_y_pred)

10.48325290009115

In [46]:
mean_squared_error(test_y, test_y_pred)

14.531581917722567

In [47]:
r2_score(train_y, train_y_pred)

0.820018681848257

In [48]:
r2_score(test_y, test_y_pred)

0.5505537907484992

In [49]:
'''
훈련 데이터에서는 R² 점수가 높지만 평가용 데이터에서는 R² 점수가 낮다면,
이는 모델이 훈련 데이터에 너무 과적합(overfitting)되었을 가능성 있
'''

'\n훈련 데이터에서는 R² 점수가 높지만 평가용 데이터에서는 R² 점수가 낮다면, \n이는 모델이 훈련 데이터에 너무 과적합(overfitting)되었을 가능성 있\n'